<a href="https://colab.research.google.com/github/arriyan-aa/AI-Hand-Computer-Vision/blob/model-setup/cv_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b>AI-Hand-Computer-Vision </b> </center> </h1>

Distinguishing between AI-generated hands vs. real hands. Our project aims to address the issue of AI in art and other media. This is an important problem because the use of AI generated material in art and media is becoming increasingly more common and has brought up discussions of ethics and copyright. This model would help distinguish real human hands versus AI generated ones.

# Libraries

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install numpy

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

YOLOv8 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLOv8 Predict Docs](https://docs.ultralytics.com/modes/train/).


if CLI the format should be:

    yolo TASK MODE ARGS

  Where:

    TASK (optional) is one of (detect, segment, classify, pose)


    MODE (required) is one of (train, val, predict, export, track)


    ARGS (optional) are arg=value pairs like imgsz=640 that override defaults.


Default ARG values are defined on this page from the cfg/defaults.yaml file.


In [2]:
%matplotlib inline
from PIL import Image

from ultralytics import YOLO
model = YOLO("yolo11n-cls.pt")

# Training

Train YOLOv8 on Detect, Segment, Classify and Pose datasets. See YOLOv8 Train Docs for more information.

In [ ]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

In YOLO (You Only Look Once), a YAML file is used for configuration and setup. It specifies parameters such as paths to datasets, model architecture, training hyperparameters, and class names. The YAML file is essential for defining how the YOLO model should be trained and what it should detect, making it an integral part of customizing the YOLO model for specific object detection tasks.

In [7]:
# Setting up yaml file
yaml_content = """
path: OneDrive/Documents/SFU SIAT/IAT360/ComputerVision Proj/hand-data # dataset root dir
train: train  # train images (relative to 'path')
val: test  # validation images (relative to 'path')
# test:  # test images (optional)

# Classes
names:
  0: ai
  1: real
"""

with open('hand_data.yaml', 'w') as f:
    f.write(yaml_content)

print("YAML file 'hand_data.yaml' created!.")

YAML file 'hand_data.yaml' created!.


In [ ]:
# Training YOLOv8n on hand dataset for 3 epochs
#!yolo train model=yolov8n.pt data=coco8.yaml epochs=3 imgsz=640

!yolo train model=yolo11n-cls.pt data=hand_data.yaml epochs=3 imgsz=640
